<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Final%20DNN%20Code%20Examples/Imagenette/Imagenette%20-%20TFDS%20Color%20Image%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imagenette - TFDS Color Image Example

This notebook demonstrates the **Universal ML Workflow** for multi-class image classification using TensorFlow Datasets (TFDS).

## Learning Objectives

By the end of this notebook, you will be able to:
- Load image datasets from **TensorFlow Datasets (TFDS)**
- Preprocess color images: Resize -> Grayscale -> Flatten -> Normalize
- Handle 10-class image classification
- Use **Top-N Accuracy** as an additional evaluation metric
- Apply Hyperband for hyperparameter tuning on image data

---

## Dataset Overview

| Attribute | Description |
|-----------|-------------|
| **Source** | [TensorFlow Datasets - imagenette/160px](https://www.tensorflow.org/datasets/catalog/imagenette) |
| **Problem Type** | Multi-Class Classification (10 classes) |
| **Classes** | Tench, English springer, Cassette player, Chain saw, Church, French horn, Garbage truck, Gas pump, Golf ball, Parachute |
| **Data Balance** | Nearly Balanced |
| **Original Size** | 160x160 color images |
| **Preprocessing** | Resize to 64x64 -> Grayscale -> Flatten (4096 features) |

---

## 1. Defining the Problem and Assembling a Dataset

**Problem:** Classify images of 10 different objects from the Imagenette dataset - a smaller subset of ImageNet designed for faster experimentation.

**Why Imagenette?** ImageNet has 1000 classes and millions of images, making it slow to iterate on. Imagenette provides a 10-class subset that's large enough to be challenging but small enough for rapid prototyping.

**Why TFDS?** TensorFlow Datasets provides easy access to common ML datasets with consistent APIs, automatic caching, and preprocessing utilities.

## 2. Choosing a Measure of Success

- **Top-1 Accuracy:** Did the model's top prediction match the true class?
- **Top-N Accuracy:** Was the true class in the model's top N predictions? (Useful for 10-class problems)
- **Precision, Recall, AUC:** Per-class and overall performance

## 3. Deciding on an Evaluation Protocol

Standard hold-out test set + validation set + K-fold cross-validation for hyperparameter tuning.

## 4. Preparing Your Data

### 4.1 Import Libraries and Load TFDS Dataset

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from skimage.color import rgb2gray
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow_datasets as tfds
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Keras Tuner for hyperparameter search
!pip install -q -U keras-tuner
import keras_tuner as kt

import itertools
import matplotlib.pyplot as plt

SEED = 204

tf.random.set_seed(SEED)
np.random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')

In [280]:
DATASET = 'imagenette/160px'
RESIZE = (64, 64, 3)
GRAY_SCALE = True

In [281]:
ds = tfds.load(DATASET, split='all', shuffle_files=True)

images, labels = [], []
for entry in ds.take(len(ds)) :
    image, label = entry['image'], entry['label']
    
    image, label = image.numpy(), label.numpy()
    
    image = resize(image, RESIZE, anti_aliasing=True)
            
    if GRAY_SCALE :
        image = rgb2gray(image)
        
    images.append( image )
    labels.append( label )

In [86]:
# list of images to numpy array
X = np.array(images)

# flatten 2D image array to 1D array
X = X.reshape( (X.shape[0], -1) )

In [87]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# label -> one-hot encoded vector
y = np_utils.to_categorical(label_encoder.transform(labels))

In [88]:
TEST_SIZE = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, stratify=labels, 
                                                    random_state=SEED, shuffle=True)

In [89]:
# normalise pixels from [0, 255] to [0, 1]
X_train, X_test = X_train/255, X_test/255

In [90]:
VALIDATION_SIZE = X_test.shape[0]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                 test_size=VALIDATION_SIZE, stratify=y_train.argmax(axis=1),
                                                 shuffle=True, random_state=SEED)

## 5. Developing a Model That Does Better Than a Baseline

Baseline for 10-class balanced problem: 10% accuracy (random guessing).

In [93]:
# number of samples per each class
counts = np.sum(y, axis=0)

# the dataset is balanced. one class is slightly less than the others, but minimal.
counts

array([1350., 1350., 1350., 1244., 1350., 1350., 1350., 1350., 1350.,
       1350.], dtype=float32)

In [94]:
baseline = max(counts) / sum(counts)

baseline

0.10079139913394057

In [106]:
INPUT_DIMENSION = X_train.shape[1]
OUTPUT_CLASSES = y_train.shape[1]

OPTIMIZER = 'RMSprop'
LOSS_FUNC = 'categorical_crossentropy'
METRICS = ['categorical_accuracy', 
           tf.keras.metrics.Precision(name='precision'), 
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc', multi_label=True)]

In [ ]:
learning_rate = 0.1

# Single Layer Perceptron: Input -> Softmax Output (no hidden layers)
slp_model = Sequential([
    Dense(OUTPUT_CLASSES, activation='softmax', input_shape=(INPUT_DIMENSION,))
], name='Single_Layer_Perceptron')

slp_model.compile(optimizer=RMSprop(learning_rate=learning_rate),
                  loss=LOSS_FUNC, metrics=METRICS)

slp_model.summary()

In [ ]:
batch_size = 128
EPOCHS = 300

In [ ]:
slp_history = slp_model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
                            validation_data=(X_val, y_val), verbose=0)
slp_val_score = slp_model.evaluate(X_val, y_val, verbose=0)[1:]

In [ ]:
print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(slp_val_score[0], baseline))
print('Precision (Validation): {:.2f}'.format(slp_val_score[1]))
print('Recall (Validation): {:.2f}'.format(slp_val_score[2]))
print('AUC (Validation): {:.2f}'.format(slp_val_score[3]))

In [157]:
def plot_training_history(history, monitors=['loss', 'AUC']) :

  # using the variable axs for multiple Axes
  fig, axs = plt.subplots(1, 2, sharex='all', figsize=(15,5))
 
  for ax, monitor in zip(axs.flat, monitors) :
    loss, val_loss = history.history[monitor], history.history['val_' + monitor]

    if monitor == 'loss' :
      monitor = monitor.capitalize()

    epochs = range(1, len(loss)+1)

    ax.plot(epochs, loss, 'b.', label=monitor)
    ax.plot(epochs, val_loss, 'r.', label='Validation ' + monitor)
    ax.set_xlim([0, len(loss)])
    ax.title.set_text('Training and Validation ' + monitor + 's')
    ax.set_xlabel('Epochs')
    ax.set_ylabel(monitor)
    ax.legend()
    ax.grid()

  _ = plt.show()

In [ ]:
plot_training_history(slp_history, monitors=['loss', 'auc'])

## 6. Scaling Up: Developing a Model That Overfits

Adding capacity to learn complex image patterns.

In [ ]:
learning_rate = 0.0025

# Multi-Layer Perceptron: Input -> Hidden(32, relu) -> Softmax Output
mlp_model = Sequential([
    Dense(32, activation='relu', input_shape=(INPUT_DIMENSION,)),
    Dense(OUTPUT_CLASSES, activation='softmax')
], name='Multi_Layer_Perceptron')

mlp_model.compile(optimizer=RMSprop(learning_rate=learning_rate),
                  loss=LOSS_FUNC, metrics=METRICS)

mlp_model.summary()

In [ ]:
mlp_history = mlp_model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
                            validation_data=(X_val, y_val), verbose=0)
mlp_val_score = mlp_model.evaluate(X_val, y_val, verbose=0)[1:]

In [ ]:
plot_training_history(mlp_history, monitors=['loss', 'auc'])

In [ ]:
print('Accuracy (Validation): {:.2f} (baseline={:.2f})'.format(mlp_val_score[0], baseline))
print('Precision (Validation): {:.2f}'.format(mlp_val_score[1]))
print('Recall (Validation): {:.2f}'.format(mlp_val_score[2]))
print('AUC (Validation): {:.2f}'.format(mlp_val_score[3]))

## 7. Regularizing Your Model and Tuning Hyperparameters

Using **Hyperband** for efficient hyperparameter tuning with a frozen architecture.

### Why Hyperband?

**Hyperband** is more efficient than grid search because it:
1. Starts training many configurations for a few epochs
2. Eliminates poor performers early
3. Allocates more resources to promising configurations

In [ ]:
# Hyperband Model Builder for Multi-Class Classification
def build_model_hyperband(hp):
    """
    Build Imagenette model with FROZEN architecture (2 layers: 64 -> 32 neurons).
    Only tunes regularization (Dropout) and learning rate.
    """
    model = keras.Sequential()
    model.add(layers.Input(shape=(INPUT_DIMENSION,)))

    # Fixed architecture: 2 hidden layers with 64 and 32 neurons
    # Layer 1: 64 neurons
    model.add(layers.Dense(64, activation='relu'))
    drop_0 = hp.Float('drop_0', 0.0, 0.5, step=0.1)
    model.add(layers.Dropout(drop_0))

    # Layer 2: 32 neurons
    model.add(layers.Dense(32, activation='relu'))
    drop_1 = hp.Float('drop_1', 0.0, 0.5, step=0.1)
    model.add(layers.Dropout(drop_1))

    # Output layer for multi-class classification
    model.add(layers.Dense(OUTPUT_CLASSES, activation='softmax'))

    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=LOSS_FUNC,
        metrics=METRICS
    )
    return model

In [ ]:
# Configure Hyperband tuner
tuner = kt.Hyperband(
    build_model_hyperband,
    objective='val_categorical_accuracy',
    max_epochs=20,
    factor=3,
    directory='imagenette_hyperband',
    project_name='imagenette_tuning'
)

# Run Hyperband search
tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=batch_size
)

In [ ]:
# Get best hyperparameters and build best model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
print(f"  Dropout Layer 1: {best_hp.get('drop_0')}")
print(f"  Dropout Layer 2: {best_hp.get('drop_1')}")
print(f"  Learning Rate: {best_hp.get('lr')}")

opt_model = tuner.hypermodel.build(best_hp)
opt_model.summary()

In [ ]:
# Train the best model
opt_history = opt_model.fit(X_train, y_train, batch_size=batch_size, epochs=50,
                            validation_data=(X_val, y_val), verbose=0)
opt_val_score = opt_model.evaluate(X_val, y_val, verbose=0)[1:]

In [255]:
def top_n_accuracy_score(y, preds, n) :
    scores = []
    for j in range(preds.shape[0]) :
        score = 1 if y[j,:].argmax() in preds[j,:].argsort()[-n:] else 0
        
        scores.append( score )
            
    return sum(scores) / len(scores)

In [ ]:
preds = opt_model.predict(X_test, verbose=0)

TOP_N = 3
within = top_n_accuracy_score(y_test, preds, TOP_N)

---

## 8. Key Takeaways

1. **TFDS simplifies data loading** - `tfds.load()` handles download, caching, and parsing
2. **Top-N Accuracy** is useful for multi-class problems (model got correct answer in top 3?)
3. **Image preprocessing:** Resize → Grayscale → Flatten sacrifices spatial info for simplicity
4. **CNNs** (not covered here) preserve spatial structure and typically perform better on images

In [272]:
print('Within Top N Accuracy (Test Data): {:.2f}'.format(within))

Within Top N Accuracy (Test Data): 0.60


---

## Appendix: Modular Functions for Multi-Class Classification

The functions below provide reusable utilities for building and training multi-class classification models.

In [ ]:
# Modular Functions for Multi-Class Classification

def build_multiclass_model(input_dim, output_classes, hidden_layers=None, 
                           dropout=None, optimizer='adam', 
                           loss='categorical_crossentropy', metrics=['accuracy'], name=None):
    """
    Build a multi-class classification model.
    
    Args:
        input_dim: Number of input features
        output_classes: Number of output classes
        hidden_layers: List of (neurons, activation) tuples for hidden layers, or None for SLP
        dropout: Dropout rate after each hidden layer (None for no dropout)
        optimizer: Keras optimizer
        loss: Loss function
        metrics: List of metrics
        name: Model name
    
    Returns:
        Compiled Keras Sequential model
    """
    model = Sequential(name=name)
    
    if hidden_layers:
        for i, (neurons, activation) in enumerate(hidden_layers):
            if i == 0:
                model.add(Dense(neurons, activation=activation, input_shape=(input_dim,)))
            else:
                model.add(Dense(neurons, activation=activation))
            if dropout:
                model.add(Dropout(dropout))
    
    # Output layer with softmax for multi-class
    if not hidden_layers:
        model.add(Dense(output_classes, activation='softmax', input_shape=(input_dim,)))
    else:
        model.add(Dense(output_classes, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model


def train_model(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=100):
    """
    Train a model and return history and validation scores.
    
    Args:
        model: Compiled Keras model
        X_train, y_train: Training data
        X_val, y_val: Validation data
        batch_size: Batch size for training
        epochs: Number of epochs
    
    Returns:
        history: Training history object
        val_score: Validation metrics (excluding loss)
    """
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                        validation_data=(X_val, y_val), verbose=0)
    val_score = model.evaluate(X_val, y_val, verbose=0)[1:]
    return history, val_score


# Example usage (commented out):
# 
# # Build an MLP for multi-class classification
# model = build_multiclass_model(
#     input_dim=INPUT_DIMENSION,
#     output_classes=OUTPUT_CLASSES,
#     hidden_layers=[(64, 'relu'), (32, 'relu')],
#     dropout=0.2,
#     optimizer=RMSprop(learning_rate=0.001),
#     loss='categorical_crossentropy',
#     metrics=METRICS,
#     name='Custom_MLP'
# )
# 
# # Train the model
# history, val_score = train_model(model, X_train, y_train, X_val, y_val,
#                                  batch_size=128, epochs=100)
# 
# # Print results
# print(f'Accuracy: {val_score[0]:.2f}')